**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Feb 18 18:28:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Apostila 06 - Processos - Planejamento.gdoc'
'Apostila 07 - Execução.gdoc'
 atividade_propagação_erros.gdoc
'atividade - revisão 1.pdf'
'atividade - revisão 2.pdf'
'atvd avaliativa 1.pdf'
'atvd avaliativa 2.pdf'
'atvd avaliativa 4.pdf'
 aula_01_conceitos.gdoc
'aula_03_cripto_intro (1).gdoc'
 aula_03_cripto_intro.gdoc
'backing track (1).zip'
'backing track.zip'
 Breve_histórico_sobre_o_trabalho.gdoc
 Capa.pdf
 Classroom
 codigos.zip
'considerações sobre diferença e igualdade (1).pdf'
'considerações sobre diferença e igualdade.pdf'
'Cópia de cap Introdução.gdoc'
'Cópia de cap Metodologia.gdoc'
'Cópia de documento de visão.gdoc'
'diagrama de atividades.pdf'
'diagrama de sequencias e comunicação.pdf'
'documento 2.pdf'
'educação em direitos humanos'
'Empilhamento de bolas (1).cpp'
'Empilhamento de bolas.cpp'
 esquematico.brd
 esquematico.sch
 Estruturas_para_Sistemas_Discretos.ipynb_-_Colaboratory.pdf
 Exercicios_Analise.gdoc
 Exercicios_Co

**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15395, done.
remote: Total 15395 (delta 0), reused 0 (delta 0), pack-reused 15395
Receiving objects: 100% (15395/15395), 14.02 MiB | 14.03 MiB/s, done.
Resolving deltas: 100% (10353/10353), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [5]:
!cp cfg/yolov4.cfg cfg/yolov4_training.cfg

In [16]:
!sed -i 's/width=608/width=416/' cfg/yolov4_training.cfg
!sed -i 's/height=608/height=416/' cfg/yolov4_training.cfg
!sed -i 's/max_batches=500500/max_batches=6000/' cfg/yolov4_training.cfg
!sed -i 's/steps=400000,450000/steps=4800,5400/' cfg/yolov4_training.cfg
!sed -i 's/subdivisions=8/subdivisions=16/' cfg/yolov4_training.cfg
# !sed -i 's/max_batches = 4000/max_batches = 500200/' cfg/yolov3_training.cfg
!sed -i '968 s@classes=80@classes=4@' cfg/yolov4_training.cfg 
!sed -i '1056 s@classes=80@classes=4@' cfg/yolov4_training.cfg
!sed -i '1144 s@classes=80@classes=4@' cfg/yolov4_training.cfg
!sed -i '961 s@filters=255@filters=27@' cfg/yolov4_training.cfg # filtros = (n_classes + 5) * 3
!sed -i '1049 s@filters=255@filters=27@' cfg/yolov4_training.cfg
!sed -i '1137 s@filters=255@filters=27@' cfg/yolov4_training.cfg
!cat cfg/yolov4_training.cfg

[net]
batch=64
subdivisions=16
# Training
#width=416
#height=416
width=416
height=416
channels=3
momentum=0.949
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.0013
burn_in=1000
max_batches = 500500
policy=steps
steps=4800,5400
scales=.1,.1

#cutmix=1
mosaic=1

#:104x104 54:52x52 85:26x26 104:13x13 for 416

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=mish

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[route]
layers = -2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=32
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[

In [7]:
!echo -e "carro\nmoto\ncaminhao\nonibus" > data/obj.names
!echo -e 'classes= 4\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# %cd obj
!cat data/obj.names
# !ls
# %cd darknet
# %cd ..
# !rm -r obj

carro
moto
caminhao
onibus


In [ ]:

# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-02-07 13:53:28--  https://doc-14-44-docs.googleusercontent.com/docs/securesc/rho9ksbgvh6j1fuci4qkvuon810q0ee7/3qqgg53cogq03ilrhmfrjm2jig4irues/1644241950000/07492844120044062954/07492844120044062954/1IZeBBfE9_qGisbCs12IL9NCZVIrk-Ck0?e=download
Resolving doc-14-44-docs.googleusercontent.com (doc-14-44-docs.googleusercontent.com)... 142.251.6.132, 2607:f8b0:4001:c5a::84
Connecting to doc-14-44-docs.googleusercontent.com (doc-14-44-docs.googleusercontent.com)|142.251.6.132|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://docs.google.com/nonceSigner?nonce=he4ig1ut8qks2&continue=https://doc-14-44-docs.googleusercontent.com/docs/securesc/rho9ksbgvh6j1fuci4qkvuon810q0ee7/3qqgg53cogq03ilrhmfrjm2jig4irues/1644241950000/07492844120044062954/07492844120044062954/1IZeBBfE9_qGisbCs12IL9NCZVIrk-Ck0?e%3Ddownload&hash=1145ilt1qki0gegbccpqjga5d78agi05 [following]
--2022-02-07 13:53:28--  https://docs.google.com/nonceSigner?nonce=he4ig1ut8qks2&continue=https:

**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [11]:
!unzip /mydrive/yolov3/images_3200.zip -d data/obj

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: data/obj/img_0700.bmp   
  inflating: data/obj/img_0700.txt   
  inflating: data/obj/img_0701.bmp   
  inflating: data/obj/img_0701.txt   
  inflating: data/obj/img_0702.bmp   
  inflating: data/obj/img_0702.txt   
  inflating: data/obj/img_0703.bmp   
  inflating: data/obj/img_0703.txt   
  inflating: data/obj/img_0704.bmp   
  inflating: data/obj/img_0704.txt   
  inflating: data/obj/img_0705.bmp   
  inflating: data/obj/img_0705.txt   
  inflating: data/obj/img_0706.bmp   
  inflating: data/obj/img_0706.txt   
  inflating: data/obj/img_0707.bmp   
  inflating: data/obj/img_0707.txt   
  inflating: data/obj/img_0708.bmp   
  inflating: data/obj/img_0708.txt   
  inflating: data/obj/img_0709.bmp   
  inflating: data/obj/img_0709.txt   
  inflating: data/obj/img_0710.bmp   
  inflating: data/obj/img_0710.txt   
  inflating: data/obj/img_0711.bmp   
  inflating: data/obj/img_0711.txt   
  inflating: data/obj/img_071

In [12]:
import glob
images_list = glob.glob("data/obj/*.bmp")
print(images_list)

['data/obj/img_1357.bmp', 'data/obj/img_0332.bmp', 'data/obj/img_1014.bmp', 'data/obj/img_0613.bmp', 'data/obj/img_1860.bmp', 'data/obj/img_2009.bmp', 'data/obj/img_2424.bmp', 'data/obj/img_1933.bmp', 'data/obj/img_2237.bmp', 'data/obj/img_3051.bmp', 'data/obj/img_1556.bmp', 'data/obj/img_2085.bmp', 'data/obj/img_2876.bmp', 'data/obj/img_1107.bmp', 'data/obj/img_0926.bmp', 'data/obj/img_3052.bmp', 'data/obj/img_2119.bmp', 'data/obj/img_1322.bmp', 'data/obj/img_0282.bmp', 'data/obj/img_3184.bmp', 'data/obj/img_1508.bmp', 'data/obj/img_2071.bmp', 'data/obj/img_0409.bmp', 'data/obj/img_2213.bmp', 'data/obj/img_2706.bmp', 'data/obj/img_2437.bmp', 'data/obj/img_1403.bmp', 'data/obj/img_1278.bmp', 'data/obj/img_0092.bmp', 'data/obj/img_0655.bmp', 'data/obj/img_1192.bmp', 'data/obj/img_0862.bmp', 'data/obj/img_1075.bmp', 'data/obj/img_3078.bmp', 'data/obj/img_0744.bmp', 'data/obj/img_0710.bmp', 'data/obj/img_1956.bmp', 'data/obj/img_1147.bmp', 'data/obj/img_0964.bmp', 'data/obj/img_2797.bmp',

In [14]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training

!./darknet detector train data/obj.data cfg/yolov4_training.cfg /mydrive/yolov3/yolov4.conv.137 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.767071), count: 6, class_loss = 0.606137, iou_loss = 0.222676, total_loss = 0.828813 
 total_bbox = 1802171, rewritten_bbox = 0.180338 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.817515), count: 27, class_loss = 1.026195, iou_loss = 92.016510, total_loss = 93.042702 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.853317), count: 18, class_loss = 0.015488, iou_loss = 9.577380, total_loss = 9.592868 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.845959), count: 9, class_loss = 0.298398, iou_loss = 0.409173, total_loss = 0.707571 
 total_bbox = 1802225, rewritten_bbox = 0.180333 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.783227), count: 68, class_loss = 4.149930, iou_loss = 205.547852, total_loss = 209